# Homework 1

The goal of this assignment was, given a set of $n$ DNA sequences, to find shortest $k$ for which there exists a set of probes of length $k$ uniquely hybridizes with each given sequence in the set.

In [24]:
from Bio import SeqIO

In [85]:
targets = [record for record in SeqIO.parse("yeast.fa_1", "fasta")]
targets = targets[:200]  # Due to computational constraints, let us analyze only 200 genes from our dataset

In [37]:
targets[:3]

[SeqRecord(seq=Seq('ATGAGTGATAATGGTTCCCCCGCGGTTCTTCCAAAAACCGAATTTAATAAATAC...TAG', SingleLetterAlphabet()), id='YPR161C', name='YPR161C', description='YPR161C <unknown description>', dbxrefs=[]),
 SeqRecord(seq=Seq('ATGAGCTTATCACCACACGTAGAAAATGCTTCCATTCCCAAGGGGAGTACCCCG...TAG', SingleLetterAlphabet()), id='YOL138C', name='YOL138C', description='YOL138C <unknown description>', dbxrefs=[]),
 SeqRecord(seq=Seq('ATGGTACAAGAACAGGCAATTTTGAGCTGCATTGAGCAGACTATGGTTGCTGAT...TGA', SingleLetterAlphabet()), id='YDR395W', name='YDR395W', description='YDR395W <unknown description>', dbxrefs=[])]

Our candidate probes are $k$-mers extracted from each sequence. 

In [38]:
def get_kmers(sequence, k):
    kmers = set()
    for i in range(len(sequence)-k):
        kmers.add(sequence[i:i+k].seq)
    return kmers

In [39]:
def get_candidate_probes(targets, k):
    candidate_probes = set()
    for seq in targets:
        for kmer in get_kmers(seq, k):
            candidate_probes.add((kmer, seq.id))
    return candidate_probes

In [40]:
list(get_candidate_probes(targets, 10))[:10]

/Users/tomek/.virtualenvs/x/lib/python3.6/site-packages/Bio/Seq.py:155: BiopythonWarning: Biopython Seq objects now use string comparison. Older versions of Biopython used object comparison. During this transition, please use hash(id(my_seq)) or my_dict[id(my_seq)] if you want the old behaviour, or use hash(str(my_seq)) or my_dict[str(my_seq)] for the new string hashing behaviour.
  "the new string hashing behaviour.", BiopythonWarning)


[(Seq('AAAACATGAT', SingleLetterAlphabet()), 'YFL009W'),
 (Seq('GACTTATGAA', SingleLetterAlphabet()), 'YDR034C-C'),
 (Seq('TCGATAATGC', SingleLetterAlphabet()), 'YBR208C'),
 (Seq('TGACCGGCTG', SingleLetterAlphabet()), 'YBR270C'),
 (Seq('CACGTTCAGA', SingleLetterAlphabet()), 'YOR323C'),
 (Seq('CAATGTCGCA', SingleLetterAlphabet()), 'YDL167C'),
 (Seq('AAACCATTGA', SingleLetterAlphabet()), 'YPL118W'),
 (Seq('CCCATTTATA', SingleLetterAlphabet()), 'YCL008C'),
 (Seq('ATATATCTTT', SingleLetterAlphabet()), 'YKR054C'),
 (Seq('ATGATGGTGA', SingleLetterAlphabet()), 'YGR097W')]

However, a probe is only good when uniquely pointing to one sequence in our set. We will discard duplicates by lexicographically sorting our candidates and saving only those probes that have non-identical neighbours. 

In [41]:
def get_unique_probes(candidate_probes):
    probes = sorted(candidate_probes, key=lambda pair: pair[0])
    unique_probes = []
    for i in range(1, len(probes)-1):
        if probes[i][0] != probes[i-1][0] and probes[i][0] != probes[i+1][0]:
            unique_probes.append(probes[i])
    return unique_probes

Finally, we'd like to find the smallest $k$ for which there is a probe set of length $k$ that covers entire sequence set.

In [81]:
def get_sequence_set_coverage(probes, targets): 
    return len(set(target for probe, target in probes))/len(targets)

In [82]:
def find_k(targets):
    k = 1
    while True:
        probes = get_unique_probes(get_candidate_probes(targets, k))
        coverage = get_sequence_set_coverage(probes, targets)
        if coverage >= 1:
            return k
        else:
            print(f'k = {k} failed with coverage {coverage*100}%, keeping searching...')
            k += 1

In [86]:
%%time
find_k(targets)

/Users/tomek/.virtualenvs/x/lib/python3.6/site-packages/Bio/Seq.py:155: BiopythonWarning: Biopython Seq objects now use string comparison. Older versions of Biopython used object comparison. During this transition, please use hash(id(my_seq)) or my_dict[id(my_seq)] if you want the old behaviour, or use hash(str(my_seq)) or my_dict[str(my_seq)] for the new string hashing behaviour.
  "the new string hashing behaviour.", BiopythonWarning)


k = 1 failed with coverage 0.0%, keeping searching...
k = 2 failed with coverage 0.0%, keeping searching...
k = 3 failed with coverage 0.0%, keeping searching...
k = 4 failed with coverage 0.0%, keeping searching...
k = 5 failed with coverage 0.0%, keeping searching...
k = 6 failed with coverage 0.0%, keeping searching...
k = 7 failed with coverage 36.5%, keeping searching...
k = 8 failed with coverage 99.5%, keeping searching...
CPU times: user 2min 43s, sys: 395 ms, total: 2min 44s
Wall time: 2min 44s


9